# Preprocessing of information

In [ ]:
# folder
root = "/workspace/tggate"

# seed for control sampling
seed=24771

# path for data folders
ORIGINAL_DIR = f"{root}/data/original"
PROCESSED_DIR = f"{root}/data/processed"

In [ ]:
import sys
import copy
from collections import defaultdict
import datetime
import pandas as pd
import numpy as np


In [ ]:
# load
info = pd.read_csv(f"{root}/data/tggate_info_ext.csv")
lst_finding = [line.rstrip() for line in open(f"{root}/data/processed/finding_lst.txt", "r")]

In [ ]:
# preprocessing
lst_finding = list(set(lst_finding)&set(info.columns))
test_info = info[(info["DOSE_LEVEL"].isin(["Middle","High"])) & (info["SACRI_PERIOD"].isin(["4 day", "8 day", "15 day", "29 day"]))]
early_info = test_info[test_info["SACRI_PERIOD"].isin(["4 day", "8 day"])]
late_info = test_info[test_info["SACRI_PERIOD"].isin(["15 day", "29 day"])]

In [ ]:
# set labels
set_compounds_late=set(late_info["COMPOUND_NAME"].tolist())
compound_label = {}
for com in late_info["COMPOUND_NAME"].unique():
    compound_label[com] = late_info[late_info["COMPOUND_NAME"] == com][lst_finding].mean().to_numpy() > 0
early_label = np.array([compound_label.get(com, [np.nan]*len(lst_finding)) for com in early_info["COMPOUND_NAME"].to_numpy()])
df_y = pd.DataFrame(early_label, columns=lst_finding)
df_y["INDEX"]=early_info.index.tolist()
df_y["FOLD"]=early_info["FOLD"].tolist()
df_y = df_y.dropna()

In [ ]:
# early information
early_info["INDEX"]=early_info.index.tolist()
early_info = early_info.loc[:,lst_finding+["INDEX"]+["FOLD"]]

In [ ]:
# export
df_y.to_csv(f"{root}/data/processed/prognosis.csv")
early_info.to_csv(f"{root}/data/processed/prognosis_early.csv")

# Evaluate

# Plot